In [2]:
import pennylane as qml

## TSP problem

In [1]:
from CEM_optimizer.CEM import CEM
from CEM_optimizer.TSP import QAOA_TSP

In [4]:
tsp = QAOA_TSP(
    number_of_cities=3,
    number_of_layers=6,
    optimization_steps=70,
    optimizer=qml.AdamOptimizer(
        stepsize=0.01,
        beta1=0.9,
        beta2=0.99
    )
)
cem = CEM(
    problem=tsp,
    epochs=10,
    samples_per_epoch=100,
    elite_frac=0.1,
    process=6
)

cem.minimize(init_weights=[1, 1, 1])

Process ForkPoolWorker-2:
Process ForkPoolWorker-1:
Process ForkPoolWorker-5:
Process ForkPoolWorker-3:
Process ForkPoolWorker-4:


KeyboardInterrupt: 

## Knapsack problem

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from CEM_optimizer.Knapsack import Knapsack, QAOA_Knapsack
from CEM_optimizer.CEM import CEM

In [3]:
knapsack = Knapsack(max_weight=1)
knapsack.set_knapsack([(1, 2), (1, 1)])

In [4]:
knapsack_qaoa = QAOA_Knapsack(knapsack, number_of_layers=3, optimization_steps=200)

In [5]:
cem = CEM(
    problem=knapsack_qaoa,
    epochs=10,
    samples_per_epoch=100,
    elite_frac=0.1,
    process=6
)

cem.minimize(init_weights=[1, 1])

Epoch 1	Average Elite Score: -1.4456714281694703	Average Score: -0.5944918110871544
[-0.18952356489511946, 1.2415183281974593] with reward: -1.8511878680498974
Epoch 2	Average Elite Score: -1.6281217350837587	Average Score: -0.671089663979994
[-1.4015475938476776, -0.5453341555179569] with reward: -1.8511878680498974
Epoch 3	Average Elite Score: -1.6141617565110262	Average Score: -0.731145464680124
[-0.3516309911628084, 1.4331479331170114] with reward: -1.8511878680498974
Epoch 4	Average Elite Score: -1.6772251078095155	Average Score: -1.1570366806493466
[-0.3761218319363802, 1.465820699967507] with reward: -1.8511878680498974
Epoch 5	Average Elite Score: -1.646874833025904	Average Score: -1.1673266725350555
[-0.3761218319363802, 1.465820699967507] with reward: -1.8511878680498974
Epoch 6	Average Elite Score: -1.6687105779959968	Average Score: -1.571904216220563
[-0.4432887217903053, 1.5601557101692505] with reward: -1.8511878680498974
Epoch 7	Average Elite Score: -1.6354578433818001	A

KeyboardInterrupt: 

In [6]:
knapsack_qaoa.print_results([-0.6696475892471498, 2.4849461246461613])

Key: 101 with probability 0.3158280268884926   | correct: True, value: 2
Key: 111 with probability 0.18057603922072862   | correct: False
Key: 001 with probability 0.13888447632383277   | correct: False
Key: 000 with probability 0.12594371574502827   | correct: True, value: 0
Key: 100 with probability 0.10076734150531425   | correct: False
Key: 011 with probability 0.04981876546514335   | correct: True, value: 1
Key: 110 with probability 0.04974280847923131   | correct: False
Key: 010 with probability 0.03843882637222737   | correct: False


In [66]:
cost_operator = knapsack_qaoa._create_cost_operator((1, 3))
def qaoa_layer(gamma, beta):
    qml.qaoa.cost_layer(gamma, cost_operator)

def hadamard_layer(wires):
    for i in range(wires):
        qml.Hadamard(i)

def circuit(params, n_layers=2):
    hadamard_layer(3)
    qml.layer(qaoa_layer, n_layers, params[0], params[1])

dev = qml.device("default.qubit", wires=3)


@qml.qnode(dev)
def cost_function(params):
    circuit(params, n_layers=1)
    x = qml.expval(cost_operator)
    return x

# fig, ax = qml.draw_mpl(cost_function, expansion_strategy="device")([[1], [2]])
# fig.show()
print(qml.draw(cost_function, expansion_strategy="device")([[1], [2]]))

range(0, 2)
[2]
2 0
2 1
0: ──H──MultiRZ(5.00)─╭MultiRZ(0.50)─╭MultiRZ(-0.50)─────────────────┤ ╭<𝓗>
1: ──H──MultiRZ(2.00)─╰MultiRZ(0.50)─│───────────────╭MultiRZ(-0.50)─┤ ├<𝓗>
2: ──H──MultiRZ(1.00)────────────────╰MultiRZ(-0.50)─╰MultiRZ(-0.50)─┤ ╰<𝓗>
